In [1]:
import pandas as pd

Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [2]:
#replace ".."with .100
Train=Train.replace('..', .210)  
Test=Test.replace('..', 0.210)  

In [3]:
#Unique country for iteration
unique = Train['Country Name'].unique()

In [4]:
#trainig and inferencing with LogisticRegression and ARIMA
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
j=[]
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
    Xtrain
    Ytrain=transposed_df[11]
    Ytrain
    logistic_regression = LogisticRegression()
    try:
        logistic_regression.fit(Xtrain,Ytrain)
        fgt = Test[Test['Country Name'] == i].reset_index(drop=True)
        columns_to_drop = [ 'Country Name','Indicator']
        fgt = fgt.drop(columns_to_drop, axis=1)
        transposed_df = fgt.transpose()
        #print(transposed_df)
  
        Xtest = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
        forecast = logistic_regression.predict(Xtest.astype(float))
        
        model = ARIMA(Ytrain.astype(float), order=(0,1,0))  # Adjust order as needed
        model_fit = model.fit()

    
        forecast2 = model_fit.forecast(steps=16)
        
        row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


        new_values = [i, forecast[0], forecast[1],forecast[2], forecast[3], forecast[4],forecast2[30]]
        sample_submission.loc[row_index] = new_values
    except:
        j.append(i)
    

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. for

In [5]:
#trainig and inferencing with LogisticRegression and ARIMA
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout


k=0
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,9,10]].astype(float)
    xt=Xtrain.values
    Ytrain=transposed_df[11].astype(float) 
    yt=Ytrain.values
    
    if k == 0:
        k+=1
        model = Sequential([
              LSTM(64, return_sequences=True, input_shape=(Xtrain.shape[1], 1)),
              Dropout(0.2),  # Adding dropout for regularization
              LSTM(64, return_sequences=True),
              Dropout(0.2),
              LSTM(64),  # Last LSTM layer without return sequences
              Dropout(0.2),
              Dense(32, activation='relu'),  # Adding a dense layer for additional processing
              Dense(1)  # Output layer
              ])
        model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
        Xtrain = xt.reshape((xt.shape[0], xt.shape[1], 1))
        model.fit( Xtrain,  yt, epochs=2, batch_size=2)
        model.save("/kaggle/working/my_lstm_model.h5")
    else:
        loaded_model = tf.keras.models.load_model("/kaggle/working/my_lstm_model.h5")
        loaded_model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
        loaded_model.fit(Xtrain, yt, epochs=2, batch_size=2)
        loaded_model.save("/kaggle/working/my_lstm_model.h5")

2024-04-01 09:33:21.834727: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 09:33:21.834870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 09:33:22.024381: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0422
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0159
Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.4066
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2117
Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 2.1002
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2688 
Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 6.8275
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5002
Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 40.7840
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 24.2251
Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 7.2608
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3565
Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 16.3171
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.7616
Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.2058
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1419
Epo

In [6]:
#predicting and forecasting unknown Co2 Value
for i in j:
    fg = Test[Test['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    XTest = transposed_df[[0,1, 2,3,4,9,10]]
    loaded_model = tf.keras.models.load_model("/kaggle/working/my_lstm_model.h5")
   
    forecast = loaded_model.predict(XTest.astype(float))
    df = pd.DataFrame(forecast)
    model2 = ARIMA(df[0], order=(0,1,0))  # Adjust order as needed
    model_fit = model2.fit()

    
    forecast2 = model_fit.forecast(steps=16)
        
    row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


    new_values = [i, forecast[0][0], forecast[1][0],forecast[2][0], forecast[3][0], forecast[4][0],forecast2[14]]
    sample_submission.loc[row_index] = new_values


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
 sample_submission.head(15)

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,0.298087813,0.298087813,0.298087813,0.298087813,0.298087813,0.297972
1,Albania,1.665218525,1.665218525,1.665218525,1.665218525,1.665218525,1.665219
2,Algeria,3.951960939,3.951960939,3.951960939,3.951960939,3.951960939,3.951961
3,American Samoa,4.092412,4.095113,4.097698,4.10032,4.10267,4.102670
4,Andorra,6.890389929,6.890389929,6.890389929,6.890389929,6.890389929,6.485769
5,Angola,1.125185364,1.125185364,1.125185364,1.125185364,1.125185364,1.125185
6,Antigua and Barbuda,5.450239602,5.450239602,5.450239602,5.450239602,5.450239602,5.450240
7,Argentina,4.301913806,4.301913806,4.301913806,4.301913806,4.301913806,4.301914
8,Armenia,1.85625279,1.85625279,1.85625279,1.85625279,1.85625279,1.856253
9,Aruba,4.379879,4.380135,4.380341,4.380549,4.382102,4.382102


In [8]:
sample_submission.to_csv('submission.csv', index=False)